# 3 - Scenario Objects
***
Scenario Objects are an container for all parameters required to run a pvdeg job. The object is meant
to be your initial step in running a pvdeg operation as it is capable of initializing your gids,
module material, mounting type, and other parameters.

Usage:

The scenario object is designed to contain the following information
- Scenario Name: Used in file paths and result file names
- Scenario Location: A path the .csv file containing all gids for the given scenario
- Modules: A list of modules to test in the scenario
    - Module Name: a unique name for the module
    - Racking Type: PVLIB racking type, used to generate module temperature parameters
    - Material: the name of the backsheet material (see materials.json) which stores all parameters for Solubility, Permeability, and Diffusivity

The Scenario is constucted as follows:

1. initialize the scenario with a name
2. add location data
3. add module data
4. View Scenario Dictionaries
5. Running a pvdeg Job with a scenario

### This is a **work in progress**

In [2]:
import pvdeg 

### 1. Initialize the Scenario Object
It is recommended that you provide a unique identifier to the Scenario, as this name will be used
when creating files and folders for the job.

If a name is not provided, it will be generated from the current date in the format "ddmmyy"

In [3]:
test_case = pvdeg.Scenario(name='test_01')

### 2. Add Location Data
Currently, the `Scenario.addLocation` function is intended to generate the gids and gids.csv
file used in the pvdeg job running with GAPs. By default, it will use the HPC's NSRDB file.

Only one location file may be added to a scenario using one of the 3 following methods

- pass NSRDB Region and Region Column
- pass Latitude and Longitude, or list of lat and long pairs
- pass a single GID or list of GIDs

In [ ]:
test_case.addLocation(region='Colorado',region_col='state')

or ...

In [ ]:
test_case.addLocation(lat_long=(39.7,-105))
# or
test_case.addLocation(lat_long=[ (39.7,-105), (38,-104), (37,-103) ])

or ...

In [4]:
test_case.addLocation(gids=[389874,389875, 389876, 389877, 389878])

Location Added - gids_test_01.csv


If the scenario instance already has a gid list, the function will report the location only.

In [3]:
test_case.addLocation()

Location Added - gids_test_01.csv


### 3.a. Add test modules
Add a module to test in the pvdeg job for all specified locaiton in the project_points file.
This requires a module name, and racking type (see PVLIB SAPM Temperature Model Params)

`Scenario.addModule` currently accepts 1 module at a time, though it can be used to add several
modules to the Scenario Object. The intent is to have each module type tested in the pvdeg Job.

If one attempts to add a module with name already in the Scenario instance, it will warn the user,
then replace the module parameters.

In [5]:
# add a module
test_case.addModule(module_name='test_module_01',
                    racking='open_rack_glass_polymer',
                    material='EVA')

# add another module
test_case.addModule(module_name='test_module_02',
                    racking='insulated_back_glass_polymer',
                    material='Helioseal 101')

# overwrite existing module
test_case.addModule(module_name='test_module_02',
                    racking='insulated_back_glass_polymer',
                    material='Tedlar')

Module "test_module_01" added.
Module "test_module_02" added.
WARNING - Module already found by name "test_module_02"
Module will be replaced with new instance.
Module "test_module_02" added.


In [ ]:
test_case.modules

## 3.b. Create New Material For Modules

If you're module is not in the list (see materials.json) you may add a custom module. The only
required parameters are

Name, Alias, Ead, Eas, So

In [ ]:
test_case.addModule(module_name='test_module_03',
                    racking='open_rack_glass_polymer',
                    material='not_a_material')

In [ ]:
test_case.add_material(name='not_a_material',alias='dlt_me',
                       Ead=999,Eas=999,So=999)

In [ ]:
test_case.addModule(module_name='test_module_03',
                    racking='open_rack_glass_polymer',
                    material='not_a_material')

## 4. View and Save Scenario Dictionaries

In [5]:
test_case.viewScenario()

Name : test_01
pipeline: []
gid file : gids_test_01.csv
test modules :
{   'module_name': 'test_module_01',
    'temperature_model': 'sapm',
    'temperature_params': {'a': -3.56, 'b': -0.075, 'deltaT': 3},
    'material_params': {   'alias': 'EVA',
                           'Fickian': True,
                           'Ead': 29.43,
                           'Do': 0.13,
                           'Eas': 32.31,
                           'So': 87.8,
                           'Eap': 61.74,
                           'Po': 97900000000.0}}
{   'module_name': 'test_module_02',
    'temperature_model': 'sapm',
    'temperature_params': {'a': -2.81, 'b': -0.0455, 'deltaT': 0},
    'material_params': {   'alias': 'PVF',
                           'Fickian': True,
                           'Ead': 30.48,
                           'Do': 0.0,
                           'Eas': -0.027,
                           'So': 0.000359,
                           'Eap': 30.21,
                           

In [6]:
test_case.exportScenario()

## 5. Running a pvdeg job with scenario objects

If you do do not pass all necessary arguments for a function, the required arguments will be
reported. 

In [12]:
param_dict = {'spectra': [1,2,3,4],
              'Ea': 5,
              'wavelengths': list(range(300,500,10))}
test_case.addFunction(func_name='degradation', func_params=param_dict)

FAILED: Requestion function degradation did not receive enough parameters
Requestion function: 
 <function Degradation.degradation at 0x7f5828902280> 
 ---
Required Parameters: 
 ['spectra', 'rh_module', 'temp_module', 'wavelengths'] 
 ---
Function has not been added to pipeline.


In [6]:
param_dict = {'spectra': [1,2,3,4],
              'rh_module': .5,
              'temp_module': [10,15,20,25],
              'wavelengths': list(range(300,500,10))}
test_case.addFunction(func_name='degradation', func_params=param_dict)

'degradation'

In [7]:
param_dict = {'poa_global': 7,
              'temp_cell': 900}
test_case.addFunction(func_name='IwaVantHoff',func_params=param_dict)

'IwaVantHoff'

In [8]:
test_case.pipeline

[{'job': 'degradation',
  'params': {'spectra': [1, 2, 3, 4],
   'rh_module': 0.5,
   'temp_module': [10, 15, 20, 25],
   'wavelengths': [300,
    310,
    320,
    330,
    340,
    350,
    360,
    370,
    380,
    390,
    400,
    410,
    420,
    430,
    440,
    450,
    460,
    470,
    480,
    490]}},
 {'job': 'IwaVantHoff', 'params': {'poa_global': 7, 'temp_cell': 900}}]

In [9]:
test_case.exportScenario()

config_test_01.json exported
